In [69]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch import optim
from lstm import LSTM

df = pd.read_csv('../../dataset_2020/user24/1599690600/eda_temp/1599696480.csv')

In [82]:
csv_file_train = pd.read_csv('/home/jh/ETRIdata/ETRItrain.csv')

In [84]:
csv_file_train['']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39060 entries, 0 to 39059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  39060 non-null  int64  
 1   ts          39060 non-null  float64
 2   action      39060 non-null  object 
 3   img_path    39060 non-null  object 
 4   eda_temp    39060 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.5+ MB


In [70]:
CFG = {
    'SEQ_LENGTH':240,
    'INPUT_DIM':2,
    'HIDDEN_DIM':10,
    'OUTPUT_DIM':1,
    'EPOCHS':100,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':100,
    'SEED':42
}

In [71]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [72]:
def build_dataset(data, seq_len):
    dataX = []
    dataY = []
    
    for i in range(len(data)-CFG['SEQ_LENGTH']):
        x = data[i:i+seq_len, :]
        y = data[i+seq_len]
        
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)


# df = df[['timestamp', 'eda', 'temp']]
df = df[['eda', 'temp']]
df = df.fillna(0)

train_size = int(len(df)*0.8)
train_set = df[0:train_size]
val_set = df[train_size-CFG['SEQ_LENGTH']:]

In [73]:
train_set

,eda,temp
0,0.285685,33.79
1,0.286966,33.79
2,0.284404,33.79
3,0.284404,33.79
4,0.285685,33.81
...,...,...
187,0.283123,34.07
188,0.283123,34.05
189,0.284404,34.05
190,0.283123,34.05


In [74]:
val_set

,eda,temp
172,0.293372,34.03
173,0.295934,34.03
174,0.293372,34.03
175,0.293372,34.03
176,0.294653,34.07
...,...,...
235,0.283123,34.09
236,0.284404,34.09
237,0.283123,34.09
238,0.283123,34.09


In [75]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_set.iloc[:,:])

train_set.iloc[:,:] = scaler_x.transform(train_set.iloc[:,:])
val_set.iloc[:,:] = scaler_x.transform(val_set.iloc[:,:])

scaler_y = MinMaxScaler()
scaler_y.fit(train_set.iloc[:,:])

train_set.iloc[:,:] = scaler_y.transform(train_set.iloc[:,:])
val_set.iloc[:,:] = scaler_y.transform(val_set.iloc[:,:])

trainX, trainY = build_dataset(np.array(train_set), CFG['SEQ_LENGTH'])
valX, valY = build_dataset(np.array(val_set), CFG['SEQ_LENGTH'])

/tmp/ipykernel_1585785/1242298558.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.iloc[:,:] = scaler_x.transform(train_set.iloc[:,:])
/tmp/ipykernel_1585785/1242298558.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set.iloc[:,:] = scaler_x.transform(val_set.iloc[:,:])
/tmp/ipykernel_1585785/1242298558.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.iloc[:,:] = scaler_y.transform(train_set.iloc[:,:]

In [76]:
trainX

array([[[0.0860471 , 0.99215574],
        [0.09082708, 0.99215574],
        [0.08126712, 0.99215574],
        ...,
        [0.0860471 , 0.99215574],
        [0.0860471 , 0.99215574],
        [0.0860471 , 0.99215574]],

       [[0.09082708, 0.99215574],
        [0.08126712, 0.99215574],
        [0.08126712, 0.99215574],
        ...,
        [0.0860471 , 0.99215574],
        [0.0860471 , 0.99215574],
        [0.08126712, 0.99215574]],

       [[0.08126712, 0.99215574],
        [0.08126712, 0.99215574],
        [0.0860471 , 0.99275914],
        ...,
        [0.0860471 , 0.99215574],
        [0.08126712, 0.99215574],
        [0.08126712, 0.99215574]],

       ...,

       [[0.11473071, 0.9993966 ],
        [0.11473071, 0.9993966 ],
        [0.11473071, 0.9993966 ],
        ...,
        [0.05586148, 0.98796831],
        [0.05586148, 0.98796831],
        [0.05586148, 0.98795011]],

       [[0.11473071, 0.9993966 ],
        [0.11473071, 0.9993966 ],
        [0.19856521, 0.9879319 ],
        .

In [77]:
trainY

array([[0.08126712, 0.99215574],
       [0.08126712, 0.99215574],
       [0.0621472 , 0.99215574],
       [0.23423766, 0.99215574],
       [0.03824357, 0.99155233],
       [0.        , 0.99155233],
       [0.        , 0.99155233],
       [0.00478371, 0.99155233],
       [0.02390363, 0.9879319 ],
       [0.01434367, 0.9879319 ],
       [0.01434367, 0.9879319 ],
       [0.02390363, 0.9879319 ],
       [0.02390363, 0.99275914],
       [0.03346359, 0.99275914],
       [0.03346359, 0.99275914],
       [0.03824357, 0.99275914],
       [0.04302355, 0.99456936],
       [0.04302355, 0.99456936],
       [0.03824357, 0.99456936],
       [0.04302355, 0.99456936],
       [0.03346359, 0.99577617],
       [0.03824357, 0.99577617],
       [0.03824357, 0.99577617],
       [0.03824357, 0.99577617],
       [0.04780353, 0.99577617],
       [0.04780353, 0.99577617],
       [0.05258724, 0.99577617],
       [0.05258724, 0.99577617],
       [0.0621472 , 0.99517276],
       [0.0621472 , 0.99517276],
       [0.

In [78]:
trainX_tensor = torch.FloatTensor(trainX).to(device)
trainY_tensor = torch.FloatTensor(trainY).to(device)

valX_tensor = torch.FloatTensor(valX).to(device)
valY_tensor = torch.FloatTensor(valY).to(device)

dataset = TensorDataset(trainX_tensor, trainY_tensor)
dataloader = DataLoader(dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, drop_last=True)

In [79]:
LSTM = LSTM(CFG['INPUT_DIM'], CFG['HIDDEN_DIM'], CFG['SEQ_LENGTH'], CFG['OUTPUT_DIM'], 1).to(device)

In [80]:
def train_model(model, train_df, epochs, lr, verbose, patience):
    criterion = nn.MSELoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_history = np.zeros(epochs)
    
    for epoch in range(epochs):
        avg_cost = 0
        total_batch = len(train_df)
        
        for batch_idx, samples in enumerate(train_df):
            x_train, y_train = samples
            model.reset_hidden_state()
            print(x_train.shape)
            outputs = model(x_train) # h(x)
            loss = criterion(outputs, y_train) # h(y)
            print('----------------')
            print(outputs)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_cost += loss/total_batch
        train_history[epoch] = avg_cost
        
        if epoch % verbose == 0:
            print('Epoch ', '%02d' % (epoch), 'Train Loss: ', '{:.4f}'.format(avg_cost))
        if (epoch % patience == 0) & (epoch != 0):
            if train_history[epoch-patience] < train_history[epoch]:
                print('\n Early Stopping')
                break
    return model.eval(), train_history

In [81]:
model, train_history = train_model(LSTM, dataloader, epochs=CFG['EPOCHS'], lr=CFG['LEARNING_RATE'], verbose=20, patience=10)

torch.Size([100, 20, 2])
----------------
tensor([[-0.1846],
        [-0.1846],
        [-0.1846],
        [-0.1847],
        [-0.1841],
        [-0.1845],
        [-0.1849],
        [-0.1852],
        [-0.1854],
        [-0.1856],
        [-0.1858],
        [-0.1859],
        [-0.1859],
        [-0.1856],
        [-0.1854],
        [-0.1853],
        [-0.1852],
        [-0.1851],
        [-0.1850],
        [-0.1849],
        [-0.1849],
        [-0.1849],
        [-0.1848],
        [-0.1848],
        [-0.1848],
        [-0.1848],
        [-0.1847],
        [-0.1847],
        [-0.1847],
        [-0.1846],
        [-0.1846],
        [-0.1846],
        [-0.1846],
        [-0.1845],
        [-0.1846],
        [-0.1846],
        [-0.1846],
        [-0.1845],
        [-0.1844],
        [-0.1844],
        [-0.1844],
        [-0.1844],
        [-0.1844],
        [-0.1844],
        [-0.1844],
        [-0.1843],
        [-0.1843],
        [-0.1843],
        [-0.1843],
        [-0.1842],
        

/home/jh/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([100, 2])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([[-0.1417],
        [-0.1418],
        [-0.1418],
        [-0.1419],
        [-0.1409],
        [-0.1417],
        [-0.1423],
        [-0.1427],
        [-0.1429],
        [-0.1430],
        [-0.1432],
        [-0.1433],
        [-0.1433],
        [-0.1430],
        [-0.1428],
        [-0.1427],
        [-0.1426],
        [-0.1424],
        [-0.1423],
        [-0.1422],
        [-0.1422],
        [-0.1422],
        [-0.1421],
        [-0.1421],
        [-0.1421],
        [-0.1420],
        [-0.1420],
        [-0.1419],
        [-0.1419],
        [-0.1418],
        [-0.1418],
        [-0.1418],
        [-0.1418],
        [-0.1417],
        [-0.1417],
        [-0.1418],
        [-0.1418],
        [-0.1417],
        [-0.1416],
        [-0.1416],
        [-0.1416],
        [-0.1416],
        [-0.1416],
        [-0.1416],
        [-0.1415],
        [-0.1415],
        [-0.1415],
        [-0.1414],
        [-0.1414],
        [-0.1413],
        [-0.1413],
        [-0.1413],
        [-0.

NameError: name 'outputs' is not defined